In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import metrics

In [2]:
dataset=pd.read_csv('./immo_data.csv')

In [3]:
dataset.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,...,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,...,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,...,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,...,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,...,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 49 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   regio1                    268850 non-null  object 
 1   serviceCharge             261941 non-null  float64
 2   heatingType               223994 non-null  object 
 3   telekomTvOffer            236231 non-null  object 
 4   telekomHybridUploadSpeed  45020 non-null   float64
 5   newlyConst                268850 non-null  bool   
 6   balcony                   268850 non-null  bool   
 7   picturecount              268850 non-null  int64  
 8   pricetrend                267018 non-null  float64
 9   telekomUploadSpeed        235492 non-null  float64
 10  totalRent                 228333 non-null  float64
 11  yearConstructed           211805 non-null  float64
 12  scoutId                   268850 non-null  int64  
 13  noParkSpaces              93052 non-null   f

In [13]:
features=['serviceCharge','heatingType','telekomUploadSpeed','totalRent']
sub_dataset=dataset[features]

In [15]:
sub_dataset

,serviceCharge,heatingType,telekomUploadSpeed,totalRent
0,245.00,central_heating,10.0,840.0
1,134.00,self_contained_central_heating,10.0,NaN
2,255.00,floor_heating,2.4,1300.0
3,58.15,district_heating,40.0,NaN
4,138.00,self_contained_central_heating,NaN,903.0
...,...,...,...,...
268845,90.00,heat_pump,10.0,910.0
268846,220.00,gas_heating,NaN,1150.0
268847,220.00,central_heating,40.0,930.0
268848,175.00,heat_pump,NaN,1015.0


In [18]:
# remove nan values in target feature
sub_dataset=sub_dataset.dropna(subset=['totalRent'])

In [19]:
sub_dataset

,serviceCharge,heatingType,telekomUploadSpeed,totalRent
0,245.0,central_heating,10.0,840.00
2,255.0,floor_heating,2.4,1300.00
4,138.0,self_contained_central_heating,NaN,903.00
6,70.0,self_contained_central_heating,2.4,380.00
7,88.0,central_heating,2.4,584.25
...,...,...,...,...
268844,80.0,gas_heating,2.4,670.00
268845,90.0,heat_pump,10.0,910.00
268846,220.0,gas_heating,NaN,1150.00
268847,220.0,central_heating,40.0,930.00


In [22]:
from sklearn.impute import SimpleImputer

In [24]:
predictor_labels=['serviceCharge','heatingType','telekomUploadSpeed']

# handle missing values
imputer=SimpleImputer(strategy='most_frequent')
sub_dataset[predictor_labels]=imputer.fit_transform(sub_dataset[predictor_labels])
sub_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228333 entries, 0 to 268848
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   serviceCharge       228333 non-null  object 
 1   heatingType         228333 non-null  object 
 2   telekomUploadSpeed  228333 non-null  object 
 3   totalRent           228333 non-null  float64
dtypes: float64(1), object(3)
memory usage: 8.7+ MB


In [25]:
from sklearn.preprocessing import OneHotEncoder

In [37]:
encoder=OneHotEncoder()
# apply one hot encoding on 'heatingType' feature
encoder.fit(sub_dataset.heatingType.values.reshape((-1,1)))
sub_dataset[list(encoder.categories_[0])]=encoder.transform(sub_dataset.heatingType.values.reshape((-1,1))).toarray()
sub_dataset.drop('heatingType',axis=1,inplace=ace=True)
sub_dataset

,serviceCharge,telekomUploadSpeed,totalRent,central_heating,combined_heat_and_power_plant,district_heating,electric_heating,floor_heating,gas_heating,heat_pump,night_storage_heater,oil_heating,self_contained_central_heating,solar_heating,stove_heating,wood_pellet_heating
0,245.0,10.0,840.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,255.0,2.4,1300.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,138.0,40.0,903.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,70.0,2.4,380.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,88.0,2.4,584.25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268844,80.0,2.4,670.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268845,90.0,10.0,910.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
268846,220.0,40.0,1150.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268847,220.0,40.0,930.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# remove outliers with zscore
std=np.std(sub_dataset[['serviceCharge','telekomUploadSpeed']],axis=0)
mean=np.mean(sub_dataset[['serviceCharge','telekomUploadSpeed']],axis=0)
zscore=(sub_dataset[['serviceCharge','telekomUploadSpeed']]-mean)/std
sub_dataset=sub_dataset[(np.abs(zscore) < 3).all(axis=1)]
sub_dataset.shape

(228125, 16)

In [51]:
from sklearn.model_selection import train_test_split

In [61]:
x=sub_dataset.drop('totalRent',axis=1)
y=sub_dataset.totalRent
# split train and test 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [62]:
from sklearn.preprocessing import StandardScaler

In [63]:
# scale ordinal features
ordinal_features=['serviceCharge','telekomUploadSpeed']
scaler=StandardScaler()
x_train[ordinal_features]=scaler.fit_transform(x_train[ordinal_features])
x_test[ordinal_features]=scaler.transform(x_test[ordinal_features])

In [64]:
x_train

,serviceCharge,telekomUploadSpeed,central_heating,combined_heat_and_power_plant,district_heating,electric_heating,floor_heating,gas_heating,heat_pump,night_storage_heater,oil_heating,self_contained_central_heating,solar_heating,stove_heating,wood_pellet_heating
38851,1.805719,0.625821,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189587,-0.602623,0.625821,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194568,1.805719,0.625821,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140235,-0.421997,0.625821,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257505,0.360714,0.625821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140830,-1.204708,0.625821,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121803,-1.036124,0.625821,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155060,-0.361789,0.625821,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172701,-0.482206,0.625821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [65]:
x_test

,serviceCharge,telekomUploadSpeed,central_heating,combined_heat_and_power_plant,district_heating,electric_heating,floor_heating,gas_heating,heat_pump,night_storage_heater,oil_heating,self_contained_central_heating,solar_heating,stove_heating,wood_pellet_heating
161067,-0.602623,-1.294642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12474,0.119880,-1.294642,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136721,-1.806793,-1.294642,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8830,-0.819373,0.625821,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229013,0.445006,0.625821,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48335,-1.204708,0.625821,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126188,-0.176828,-1.781159,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143769,-1.084291,0.625821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
56559,-0.963874,-1.294642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [82]:
class Linear_Regression:
    def __init__(self,alpha,random_state=None,method='grad_descent',max_iter=100):
        self.alpha=alpha
        self.random_state=random_state
        self.method=eval(f'self.{method}')
        self.max_iter=max_iter
    def fit(self,x,y):
        X=np.array(x)
        Y=np.array(y)
        # adding bias to X
        ones=np.ones_like(X[:,0])
        X=np.hstack((np.reshape(ones,(-1,1)),X))
        np.random.seed(self.random_state)
        self.theta_=np.random.rand(X.shape[1],1)
        Y=Y.reshape((-1,1))
        self.method(X,Y)
    
    def grad_descent(self,x,y):
        self.history_=np.array([[0,self.cost_function(x,y)]])
        last=self.cost_function(x,y)
        
        for index in range (1,self.max_iter+1):
            self.theta_=self.theta_-self.alpha*(1/x.shape[0])*(np.dot(x.T, (self.hypotsis_function(x)- y)))
            self.history_=np.vstack((self.history_, np.array([index,self.cost_function(x,y)])))
            if self.cost_function(x,y)>=last:
                self.alpha=self.alpha/2
            last=self.cost_function(x,y)
    
    def hypotsis_function(self,x):
        return x@self.theta_
    
    def cost_function(self,x,y):
        predictions = x@ self.theta_
        errors = predictions - y
        return 0.5 *(1/x.shape[0])* (errors.T@ errors)[0, 0]
    
    def normal_equation(self,x,y):
        self.theta_=np.linalg.pinv(x.T@x)@(x.T@y)
    
    def predict(self,x):
        X=np.hstack((np.reshape(np.ones_like(x[:,0]),(-1,1)),x))
        return (X@self.theta_).ravel()
    

In [83]:
model=Linear_Regression(alpha=0.1,random_state=42,max_iter=1000)
model.fit(x_train,y_train)

In [86]:
# MSE of our model on train data
pred=model.predict(x_train.values)
metrics.mean_squared_error(y_train,pred)

24334680.01762735

In [87]:
# MSE of our model on test data
pred=model.predict(x_test.values)
metrics.mean_squared_error(y_test,pred)

3628373838.197944

In [97]:
model.theta_

array([[783.18398146],
       [373.16230277],
       [  4.86713413],
       [ 34.54975101],
       [100.26810721],
       [ -6.52008139],
       [ 24.05504818],
       [261.46311662],
       [ 13.82019429],
       [158.75762303],
       [  2.5825481 ],
       [ 33.00363146],
       [ 93.96850311],
       [ 13.20496554],
       [  6.71141905],
       [ 52.4892397 ]])

In [88]:
from sklearn.linear_model import LinearRegression

In [89]:
LR=LinearRegression()
LR.fit(x_train,y_train)

LinearRegression()

In [91]:
# MSE of sklearn model on train data
metrics.mean_squared_error(y_train,LR.predict(x_train))

24334540.52064295

In [92]:
# MSE of sklearn model on test data
metrics.mean_squared_error(y_test,LR.predict(x_test))

3628346496.0389085

In [98]:
LR.coef_

array([3.72417140e+02, 4.70839506e+00, 7.58484533e+12, 7.58484533e+12,
       7.58484533e+12, 7.58484533e+12, 7.58484533e+12, 7.58484533e+12,
       7.58484533e+12, 7.58484533e+12, 7.58484533e+12, 7.58484533e+12,
       7.58484533e+12, 7.58484533e+12, 7.58484533e+12])

In [93]:
from sklearn.linear_model import Lasso,Ridge

In [95]:
# train ridge regression on data
ridge=Ridge(alpha=0.1,max_iter=1000)
ridge.fit(x_train,y_train)

Ridge(alpha=0.1, max_iter=1000)

In [96]:
# MSE of sklearn ridge model on train data
metrics.mean_squared_error(y_train,ridge.predict(x_train))

24334540.408087403

In [99]:
# MSE of sklearn ridge model on test data
metrics.mean_squared_error(y_test,ridge.predict(x_test))

3628346275.6042223

In [101]:
ridge.coef_

array([ 372.56912748,    4.7154186 ,  -58.45163401,   67.70890021,
       -101.75303253,  -36.06264722,  165.80059033,  -81.94273229,
        138.80006449, -112.55880602,  -70.28853469,   -2.97877502,
        108.30333094,  -61.31034583,   44.73362296])

In [100]:
# train lasso regression on data
lasso=Lasso(alpha=0.1,max_iter=1000)
lasso.fit(x_train,y_train)

Lasso(alpha=0.1)

In [102]:
# MSE of sklearn lasso model on train data
metrics.mean_squared_error(y_train,lasso.predict(x_train))

24334560.546804726

In [103]:
# MSE of sklearn lasso model on test data
metrics.mean_squared_error(y_test,lasso.predict(x_test))

3628352200.2974377

In [104]:
lasso.coef_

array([372.75110254,   4.66541182, -35.13204585,  78.00316543,
       -77.51975823,  -0.        , 187.37895858, -57.54236596,
       151.65730478, -69.64761302, -42.11163546,  18.70538964,
         0.        ,  -0.        ,  41.5836656 ])